In [1]:
import numpy as np
import dicom
import pandas as pd
import os
import matplotlib.pyplot as pl
from report_parser import parse_report
from PIL import Image
import tqdm
from collections import defaultdict
import pickle
import gzip
import random

from report_parser import parse_report

/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
# data stored on our server
csv_file_path = '/crimea/mimic-cxr/cache/mimic-cxr-map.csv'
images_path = '/crimea/mimic-cxr/images'
reports_path = '/crimea/mimic-cxr/reports'

image_files  = set(os.listdir(images_path))
report_files = set(os.listdir(reports_path))

print('images:  %6d' % len( image_files)) #  91664
print('reports: %6d' % len(report_files)) # 206574

images:  473064
reports: 206574


In [3]:
df =  pd.read_csv(csv_file_path, sep=',', header=0)
df = df.loc[df['dicom_is_available'],:]
df.reset_index()
print(df.shape)                                             # (473057, 4)
display(df.head())

print('unique subjects: %6d' % len(set(df['subject_id'])))  # unique subjects:  63478
print('unique     rads: %6d' % len(set(df[    'rad_id'])))  # unique     rads: 206563
print('unique   dicoms: %6d' % len(set(df[  'dicom_id'])))  # unique   dicoms: 473057

(473057, 4)


,subject_id,rad_id,dicom_id,dicom_is_available
0,70233355,52727485,3b9565f5-69ab0d33-1a9d2d1b-bb09c424-7f0243e6,True
1,70233355,52727485,8074bd10-62acdde0-3df2608b-13ca2322-09ce372c,True
2,70233355,53378012,3108d905-782ffdc0-209309e8-2413eeb4-6bfb958a,True
3,70233355,53378012,40eab5a8-31446771-08c6b024-2717a65c-41f8c74f,True
4,70233355,55587989,cef1a7ea-8c7df75c-41070128-7cdf5c89-23682e1b,True


unique subjects:  63478
unique     rads: 206563
unique   dicoms: 473057


In [8]:
positions = {}

columns = df.columns
for vals in tqdm.tqdm(df.values):
    row = dict(zip(columns,vals))
    dicom_file  = str(row['dicom_id'])+'.dcm.gz'
    report_file = str(row['rad_id'])+'.txt'
    if (dicom_file in image_files) and (report_file in report_files):
        
        # only AP images
        dicom_path = os.path.join(images_path,dicom_file)
        dicom_img = gzip.open(dicom_path)
        plan = dicom.read_file(dicom_img, stop_before_pixels=False)
        view_position = plan.ViewPosition
        
        dicom_id = dicom_file.split('.')[0]
        positions[dicom_id] = view_position


100%|██████████| 10/10 [00:00<00:00, 10.93it/s]


In [9]:
with open('/crimea/wboag/2019/cxr/view_positions.pkl', 'wb') as f:
    pickle.dump(positions, f)